In [224]:
import yfinance as yf
import pandas as pd
ticker=input('Please Enter The Ticker of the Stock')
# Request historical data for past 5 years
dataset = yf.Ticker(ticker).history(period='5y')
# Show info
print(dataset.info())
sdata=pd.DataFrame(data=dataset,)
sdata['Date']=sdata.index
sdata

Please Enter The Ticker of the Stock NVDA


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1259 entries, 2017-05-10 to 2022-05-09
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          1259 non-null   float64
 1   High          1259 non-null   float64
 2   Low           1259 non-null   float64
 3   Close         1259 non-null   float64
 4   Volume        1259 non-null   int64  
 5   Dividends     1259 non-null   float64
 6   Stock Splits  1259 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 78.7 KB
None


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
Date,,,,,,,,
2017-05-10,28.231580,30.091618,28.164884,29.960699,212909600,0.0,0.0,2017-05-10
2017-05-11,29.654397,32.218431,29.619814,31.247656,193979200,0.0,0.0,2017-05-11
2017-05-12,31.279767,32.013410,31.069803,31.591009,96262000,0.0,0.0,2017-05-12
2017-05-15,32.003533,33.201568,31.959072,33.176865,108754000,0.0,0.0,2017-05-15
2017-05-16,33.688188,33.950025,32.942195,33.794403,112070000,0.0,0.0,2017-05-16
...,...,...,...,...,...,...,...,...
2022-05-03,194.000000,198.250000,191.330002,196.020004,47575100,0.0,0.0,2022-05-03
2022-05-04,199.229996,204.000000,187.509995,203.339996,64885500,0.0,0.0,2022-05-04
2022-05-05,198.669998,199.250000,185.000000,188.440002,62633100,0.0,0.0,2022-05-05


In [206]:
#Calculating the returns based on close prices of the stock.
returns=sdata['Close'].pct_change()
returns=returns.dropna()
returns=returns.reset_index()
returns=returns['Close']
returns
returns.tail(50)

1208    0.009438
1209   -0.037236
1210    0.031824
1211   -0.020892
1212   -0.032808
1213   -0.069062
1214    0.007587
1215    0.069722
1216   -0.015469
1217   -0.024627
1218   -0.034842
1219    0.077028
1220    0.066295
1221    0.011022
1222    0.068118
1223    0.010623
1224   -0.007855
1225   -0.033554
1226    0.098151
1227   -0.016270
1228    0.019031
1229    0.015486
1230   -0.033710
1231   -0.014590
1232   -0.021036
1233    0.024259
1234   -0.052230
1235   -0.058771
1236   -0.008153
1237   -0.044985
1238   -0.051992
1239   -0.018844
1240    0.032506
1241   -0.042562
1242    0.024697
1243    0.019052
1244   -0.032255
1245   -0.060469
1246   -0.033097
1247    0.019831
1248   -0.055974
1249   -0.019853
1250    0.074233
1251   -0.062431
1252    0.053162
1253    0.003532
1254    0.037343
1255   -0.073276
1256   -0.008968
1257   -0.092369
Name: Close, dtype: float64

In [207]:
#We will not be applying the scalar function or standardizing function because LSTM  is useful in predicting based on relevant information of patterns in the past.
#By changing the scale, we will lose out on the information and using LSTM would have little use.

##splitting dataset into train and test split 90-10
import numpy as np
training_size=int(0.6*len(returns))
testing_size=int(len(returns)-training_size)
training_data=returns[1:training_size]
testing_data=returns[training_size+1:len(returns)]
training_data=np.array(training_data)
testing_data=np.array(testing_data)

testing_size

504

In [208]:

# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX,dataY = [],[]
	for i in range(len(dataset)-time_step-1):
                a = dataset[i:(i+time_step)] 
                b = dataset[i + time_step] 
                dataX.append(a)
                dataY.append(b)
	return np.array(dataX),np.array(dataY)

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(training_data,time_step)
X_test, ytest = create_dataset(testing_data,time_step)

In [209]:
# reshape input to be [samples, time steps, features] which is required for LSTM

X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [210]:
### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [211]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [212]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
11/11 [==============================] - 42s 790ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 2/100
11/11 [==============================] - 4s 352ms/step - loss: 0.0010 - val_loss: 9.7079e-04
Epoch 3/100
11/11 [==============================] - 4s 354ms/step - loss: 0.0010 - val_loss: 9.7038e-04
Epoch 4/100
11/11 [==============================] - 4s 350ms/step - loss: 9.9457e-04 - val_loss: 9.6995e-04
Epoch 5/100
11/11 [==============================] - 4s 361ms/step - loss: 9.9915e-04 - val_loss: 9.7808e-04
Epoch 6/100
11/11 [==============================] - 4s 343ms/step - loss: 0.0010 - val_loss: 9.7237e-04
Epoch 7/100
11/11 [==============================] - 4s 352ms/step - loss: 9.9600e-04 - val_loss: 9.7546e-04
Epoch 8/100
11/11 [==============================] - 4s 355ms/step - loss: 0.0010 - val_loss: 9.7360e-04
Epoch 9/100
11/11 [==============================] - 4s 347ms/step - loss: 9.9266e-04 - val_loss: 9.7547e-04
Epoch 10/100
11/11 [======================

In [213]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [214]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))

0.03146804196778804

In [215]:
### Test Data RMSE
math.sqrt(mean_squared_error(ytest,test_predict))

0.03134663766753313

In [216]:
ytest


array([-4.05634365e-02,  4.44217960e-02,  6.89018564e-03,  1.65618353e-02,
       -8.96970896e-03, -5.49163607e-03,  3.36595471e-02,  1.56422567e-03,
       -1.07383820e-02, -8.88586285e-03, -1.13456770e-02, -2.27167858e-02,
        1.09463962e-02, -8.84908882e-03, -1.21073631e-02,  1.71581156e-02,
       -3.30382851e-02,  1.94424239e-02, -5.74579203e-02,  3.14404648e-02,
       -3.76227653e-02,  3.73002692e-03,  3.48747047e-02,  5.95067858e-02,
        2.65147559e-02,  2.83896933e-02, -6.39506456e-02, -6.31476151e-02,
        5.07437946e-02,  2.88800031e-03, -1.18714005e-02,  1.64134817e-02,
       -6.88112288e-03,  4.84368335e-04,  8.56308167e-04, -2.62271529e-02,
        3.99219540e-03, -1.38697979e-02,  2.13771854e-02,  2.00232697e-03,
        1.05758943e-02, -8.58163389e-04,  1.15385010e-02, -1.06717153e-02,
        1.21117937e-02,  3.57717906e-03, -1.88691763e-02, -3.14047030e-02,
        3.20951330e-03,  3.16062354e-03,  2.27075158e-02,  3.88836081e-03,
       -8.83191062e-03,  

In [217]:
#Prediction for next 10 days
len(testing_data)
x_input=testing_data[(len(testing_data)-100):].reshape(1,-1)
x_input.shape

(1, 100)

In [218]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [219]:
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day prediction of return is {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
    



[-0.00045448]
1 day prediction of return is [[-9.608781e-05]]
2 day prediction of return is [[0.00052697]]
3 day prediction of return is [[0.00125949]]
4 day prediction of return is [[0.00196135]]
5 day prediction of return is [[0.00253431]]
6 day prediction of return is [[0.00292616]]
7 day prediction of return is [[0.00312467]]
8 day prediction of return is [[0.00314705]]
9 day prediction of return is [[0.00302891]]
10 day prediction of return is [[0.00281481]]
11 day prediction of return is [[0.0025504]]
12 day prediction of return is [[0.00227705]]
13 day prediction of return is [[0.00202835]]
14 day prediction of return is [[0.00182841]]
15 day prediction of return is [[0.00169173]]
16 day prediction of return is [[0.00162404]]
17 day prediction of return is [[0.00162326]]
18 day prediction of return is [[0.0016818]]
19 day prediction of return is [[0.0017881]]
20 day prediction of return is [[0.00192847]]
21 day prediction of return is [[0.00208868]]
22 day prediction of return i